In [1]:
from functools import partial
from psutil import cpu_count
import logging

from Freesound.data import *
from Freesound.utils import *
from Freesound.model import *
from Freesound.augmentations import *

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

### utils

In [2]:
%reload_ext autoreload
%autoreload 2

SEED = 520
seed_everything(SEED)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log", filemode="w+")

N_JOBS = cpu_count()
os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
os.environ['OMP_NUM_THREADS'] = str(N_JOBS)
DataLoader = partial(DataLoader, num_workers=N_JOBS)

In [3]:
names, y_curated_np, y_noisy_np, train_curated, train_noisy, test_df, labels = get_tables()

In [5]:
post_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    RandomParameter(RandomNoise, [[0.01, 0.1]]),
    #RandomParameter(Shift, [[2000, 32000]]),
    RandomParameter(TimeStretch, [[0.75, 1.3]]),
    ToMellSpec(n_mels=331),
    #GetMFCC(),
    PadOrClip(196),
    librosa.power_to_db,
    #Normalize(),
    #ToTensor(),
    Normalize_predef(-24.42848536693399, 13.524654994769366),
    #Normalize(),
    ToTensor(),
])

### predict

In [6]:
test_audio = load_audio(names, load=True, path='Freesound/pickles/test_audio.pkl')

In [7]:
hm = Train()

test_dataset = Dataset_Test(test_audio, names, post_transform, tta=25, preload=True)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False)

model = Kaggle2(80).cuda()

#model.load_state_dict(torch.load('models/simple_old_conv.pt'))
model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
model_dst.eval()
model_dst.load_state_dict(torch.load('models/Kaggle2_RT.pt'))
pred = hm.predict_on_test(test_loader, model_dst)

In [8]:
test_df[labels] = pred.values
test_df.to_csv('submission.csv', index=False)
test_df.head()

,fname,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,...,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,000ccb97.wav,0.000500,0.000953,0.000515,0.000309,0.000471,0.005144,0.001317,0.009013,0.013336,...,0.001144,0.001671,0.006443,0.002810,0.003575,0.001192,0.037511,0.030079,0.001964,0.016716
1,0012633b.wav,0.075347,0.000212,0.000731,0.002403,0.015941,0.002409,0.000491,0.018423,0.001363,...,0.007848,0.021295,0.002293,0.027373,0.004842,0.065864,0.017718,0.017335,0.004094,0.116587
2,001ed5f1.wav,0.001356,0.000483,0.000859,0.005510,0.011966,0.002145,0.001406,0.012574,0.004894,...,0.007784,0.000846,0.006066,0.038252,0.004244,0.003782,0.011317,0.015627,0.003168,0.015701
3,00294be0.wav,0.001722,0.000145,0.000931,0.000166,0.002210,0.003461,0.000440,0.005650,0.001314,...,0.007780,0.003040,0.018353,0.011935,0.006546,0.006160,0.012540,0.048127,0.000476,0.021321
4,003fde7a.wav,0.000091,0.001302,0.000375,0.000115,0.000236,0.000185,0.000189,0.001306,0.741229,...,0.000922,0.005134,0.001235,0.002433,0.000527,0.000879,0.001781,0.004161,0.000824,0.002702
